In [1]:
from keras import layers
from keras.models import Sequential
import numpy as np
import nltk
from nltk.corpus import reuters
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import get_file
from keras.initializers import Constant
from keras.utils import to_categorical

MAX_NUM_WORDS = 10000
MAX_SEQUENCE_LEN = 500

Using TensorFlow backend.


In [20]:
nltk.download('reuters')
nltk.download('punkt')
fileids = reuters.fileids()
fileids_test = np.array([fid for fid in fileids if fid.startswith("test")])
fileids_train = np.array([fid for fid in fileids if fid.startswith("train")])

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(reuters.raw(fid) for fid in fileids)
X_train = tokenizer.texts_to_sequences(" ".join(sent) for fid in fileids_train for sent in reuters.sents(fid))
X_test = tokenizer.texts_to_sequences(" ".join(sent) for fid in fileids_test for sent in reuters.sents(fid))

In [0]:
X_train = sequence.pad_sequences(X_train, maxlen=MAX_SEQUENCE_LEN)
X_test = sequence.pad_sequences(X_test, maxlen=MAX_SEQUENCE_LEN)

In [0]:
!wget "http://nlp.stanford.edu/data/glove.6B.zip"
!unzip "glove.6B.zip"

--2019-01-13 13:33:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-01-13 13:33:04--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip         75%[==============>     ] 623.82M  11.4MB/s    eta 20s    

In [0]:
# get glove coeff matrix
embeddings_index = {}
with open("glove.6B.100d.txt", encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

# prepare pre-learned embedding matrix
embdedding_dim = 100
word_index = tokenizer.word_index
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, embdedding_dim))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
def create_model(encoder_units=256, decoder_units=256, rnn_layer=layers.GRU):
    #embedding = layers.Embedding(num_words, embdedding_dim
    #                      , input_length=MAX_SEQUENCE_LEN
    #                      , embeddings_initializer=Constant(embedding_matrix), trainable=True)
    embedding = layers.Embedding(MAX_NUM_WORDS, 100)
    model = Sequential()
    model.add(embedding)
    model.add(layers.Bidirectional(rnn_layer(encoder_units, return_sequences=False)))
    model.add(layers.RepeatVector(MAX_SEQUENCE_LEN))
    model.add(layers.Bidirectional(rnn_layer(decoder_units, return_sequences=True)))
    model.add(layers.TimeDistributed(layers.Dense(MAX_NUM_WORDS, activation='softmax')))
    model.compile(optimizer='adam'
                  , loss='categorical_crossentropy'
                  , metrics=['accuracy'])
    return model

In [6]:
model = create_model(rnn_layer=layers.CuDNNGRU)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               549888    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 500, 512)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 500, 512)          1182720   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 500, 10000)        5130000   
Total params: 7,862,608
Trainable params: 7,862,608
Non-trainable params: 0
_________________________________________________________________


In [0]:
def data_generator(X_in, batch_size=32, shuffle=True, repeat=True):
    index = np.arange(X_in.shape[0])
    while True:
        np.random.shuffle(index)
        X = X_in[index]
        n = X.shape[0]//batch_size
        for chunk in np.split(X[:n*batch_size], n):
            yield chunk, to_categorical(chunk, MAX_NUM_WORDS)
        rest = X[n*batch_size:]
        if rest.shape[0]:
            yield rest, to_categorical(rest, MAX_NUM_WORDS)
        if not repeat:
            break

In [8]:
bs = 32
model.fit_generator(data_generator(X_train, batch_size=bs)
                    , steps_per_epoch=X_train.shape[0]//bs
                    , epochs=10)

Epoch 1/10
242/242 [==============================] - 302s 1s/step - loss: 2.4272 - acc: 0.7393
Epoch 2/10
242/242 [==============================] - 296s 1s/step - loss: 2.0578 - acc: 0.7444
Epoch 3/10
242/242 [==============================] - 297s 1s/step - loss: 1.7929 - acc: 0.7539
Epoch 4/10
242/242 [==============================] - 297s 1s/step - loss: 1.7650 - acc: 0.7552
Epoch 5/10
242/242 [==============================] - 296s 1s/step - loss: 1.7343 - acc: 0.7570
Epoch 6/10
242/242 [==============================] - 298s 1s/step - loss: 1.7195 - acc: 0.7565
Epoch 7/10
242/242 [==============================] - 297s 1s/step - loss: 1.7018 - acc: 0.7577
Epoch 8/10
242/242 [==============================] - 297s 1s/step - loss: 1.7029 - acc: 0.7568
Epoch 9/10
242/242 [==============================] - 297s 1s/step - loss: 1.6858 - acc: 0.7582
Epoch 10/10
242/242 [==============================] - 297s 1s/step - loss: 1.6819 - acc: 0.7582


In [15]:
tokenizer.sequences_to_texts(np.argmax(model.predict(X_train[:10], verbose=1), axis=2))

10/10 [==============================] - 0s 45ms/step


['the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the th

In [16]:
tokenizer.sequences_to_texts(X_train[:10])

['temporao although normal levels have not been restored smith said in its weekly review the dry period means the temporao will be late this year arrivals for the week ended february 22 were 155 221 bags of 60 kilos making a cumulative total for the season of 5 93 mln against 5 81 at the same stage last year again it seems that cocoa delivered earlier on was included in the arrivals figures smith said there is still some doubt as to how much old crop cocoa is still available as harvesting has come to an end with total bahia crop estimates around 6 4 mln bags and sales standing at almost 6 2 mln there are a few hundred thousand bags still in the hands of farmers exporters and processors there are doubts as to how much of this cocoa would be fit for export as shippers are now in obtaining bahia superior certificates in view of the lower quality over recent weeks farmers have sold a good part of their cocoa held on smith said spot bean prices rose to 340 to 350 cruzados per of 15 kilos be